In [1]:
from esmvalcore.experimental import CFG
from esmvalcore.dataset import Dataset, datasets_to_recipe
import yaml

In [2]:
session = CFG.start_session('test_session')
session

Session({'auxiliary_data_dir': PosixPath('/home/bandela/esmvaltool_extra_data'),
         'check_level': <CheckLevels.DEFAULT: 3>,
         'compress_netcdf': False,
         'config_developer_file': None,
         'config_file': PosixPath('/home/bandela/.esmvaltool/config-user.yml'),
         'download_dir': PosixPath('/home/bandela/climate_data'),
         'drs': {'CMIP3': 'ESGF',
                 'CMIP5': 'ESGF',
                 'CMIP6': 'ESGF',
                 'CORDEX': 'ESGF',
                 'obs4MIPs': 'ESGF'},
         'exit_on_warning': False,
         'extra_facets_dir': (),
         'log_level': 'info',
         'max_datasets': None,
         'max_parallel_tasks': 1,
         'max_years': None,
         'offline': True,
         'output_dir': PosixPath('/home/bandela/esmvaltool_output'),
         'output_file_type': 'png',
         'profile_diagnostic': False,
         'remove_preproc_dir': False,
         'resume_from': [],
         'rootpath': {'OBS': [PosixPath('/home/

In [3]:
ds = Dataset(short_name='tas', mip='Amon', project='CMIP5', dataset='CanESM2', ensemble='r1i1p1', exp='historical', timerange='2000/2002')
ds

Dataset:
{'dataset': 'CanESM2',
 'project': 'CMIP5',
 'mip': 'Amon',
 'short_name': 'tas',
 'ensemble': 'r1i1p1',
 'exp': 'historical',
 'timerange': '2000/2002'}

In [4]:
ds.add_ancillary(short_name='areacella', mip='fx', ensemble='r0i0p0')
ds.ancillaries

[Dataset:
 {'dataset': 'CanESM2',
  'project': 'CMIP5',
  'mip': 'fx',
  'short_name': 'areacella',
  'ensemble': 'r0i0p0',
  'exp': 'historical',
  'timerange': '2000/2002'}]

In [5]:
ds.session = session
ds.augment_facets()
print(ds)
print(ds.ancillaries)

Dataset:
{'dataset': 'CanESM2',
 'project': 'CMIP5',
 'mip': 'Amon',
 'short_name': 'tas',
 'ensemble': 'r1i1p1',
 'exp': 'historical',
 'frequency': 'mon',
 'institute': ['CCCma'],
 'long_name': 'Near-Surface Air Temperature',
 'modeling_realm': ['atmos'],
 'original_short_name': 'tas',
 'product': ['output1', 'output2'],
 'standard_name': 'air_temperature',
 'timerange': '2000/2002',
 'units': 'K'}
ancillaries:
  {'dataset': 'CanESM2',
   'project': 'CMIP5',
   'mip': 'fx',
   'short_name': 'areacella',
   'ensemble': 'r0i0p0',
   'exp': 'historical',
   'frequency': 'fx',
   'institute': ['CCCma'],
   'long_name': 'Atmosphere Grid-Cell Area',
   'modeling_realm': ['atmos', 'land'],
   'original_short_name': 'areacella',
   'product': ['output1', 'output2'],
   'standard_name': 'cell_area',
   'timerange': '2000/2002',
   'units': 'm2'}
[Dataset:
{'dataset': 'CanESM2',
 'project': 'CMIP5',
 'mip': 'fx',
 'short_name': 'areacella',
 'ensemble': 'r0i0p0',
 'exp': 'historical',
 'freque

In [6]:
ds.find_files()
print(ds.files)
for anc in ds.ancillaries:
    print(anc.files)

['/home/bandela/climate_data/cmip5/output1/CCCma/CanESM2/historical/mon/atmos/Amon/r1i1p1/v20120718/tas_Amon_CanESM2_historical_r1i1p1_185001-200512.nc']
['/home/bandela/climate_data/cmip5/output1/CCCma/CanESM2/historical/fx/atmos/fx/r0i0p0/v20120410/areacella_fx_CanESM2_historical_r0i0p0.nc']


In [7]:
#cube = ds.load()
#cube

In [8]:
#cell_area = cube.cell_measures()[0]
#cell_area

In [9]:
ds['diagnostic'] = 'diagnostic'

tas = ds.copy()
pr = ds.copy(short_name='pr')
print(yaml.safe_dump(datasets_to_recipe([tas, pr]), sort_keys=False))

diagnostics:
  diagnostic:
    variables:
      tas:
        additional_datasets:
        - mip: Amon
          project: CMIP5
          dataset: CanESM2
          ensemble: r1i1p1
          exp: historical
          timerange: 2000/2002
          ancillary_variables:
          - short_name: areacella
            mip: fx
            ensemble: r0i0p0
      pr:
        additional_datasets:
        - mip: Amon
          project: CMIP5
          dataset: CanESM2
          ensemble: r1i1p1
          exp: historical
          timerange: 2000/2002
          ancillary_variables:
          - short_name: areacella
            mip: fx
            ensemble: r0i0p0



In [10]:
ds2 = Dataset(short_name='tas', mip='Amon', project='CMIP5', dataset='CanESM2', ensemble='r1i1p1', exp='historical', timerange='*')
ds2

Dataset:
{'dataset': 'CanESM2',
 'project': 'CMIP5',
 'mip': 'Amon',
 'short_name': 'tas',
 'ensemble': 'r1i1p1',
 'exp': 'historical',
 'timerange': '*'}

In [11]:
ds2.augment_facets(session)

KeyError: 'timerange'